In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Optional, Tuple, Dict
from lifelines import KaplanMeierFitter, CoxPHFitter, NelsonAalenFitter
from sklearn.model_selection import KFold
from scipy.stats import gaussian_kde, norm
from sklearn.model_selection import train_test_split

from going_modular import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

In [5]:
dfs = pd.read_csv('/Users/Apple/projects/ALS_Digital_Twins/All_processed_data/Results/combined_other_datasets_for_modeling.csv')
dfs.columns

print(f"{dfs['Study_id'].value_counts().to_dict()}\n")

print(f"{dfs['Expt'].value_counts().to_dict()}")


{'301': 959, 'miro': 220, 'lica': 176, '302': 168, '216': 155}

{'riluzole': 1282, 'mirocals': 220, 'licals': 176}


In [6]:
miroli = dfs[dfs['Expt'] != 'riluzole']

print(f"{miroli['Study_id'].value_counts().to_dict()}\n")
miroli = miroli.drop(columns=['Study_id'])
print(miroli.info())
miroli.head(3)

{'miro': 220, 'lica': 176}

<class 'pandas.core.frame.DataFrame'>
Index: 396 entries, 0 to 395
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   subject_id        396 non-null    object 
 1   Event             396 non-null    float64
 2   Vital_capacity    396 non-null    float64
 3   Sex               396 non-null    object 
 4   Onset_site        396 non-null    object 
 5   Study_Arm         396 non-null    object 
 6   European          396 non-null    int64  
 7   Age               396 non-null    float64
 8   Diagnostic_Delay  396 non-null    float64
 9   Disease_Duration  396 non-null    float64
 10  TRICALS           396 non-null    float64
 11  Expt              396 non-null    object 
dtypes: float64(6), int64(1), object(5)
memory usage: 40.2+ KB
None


,subject_id,Event,Vital_capacity,Sex,Onset_site,Study_Arm,European,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Expt
0,P01001,0.0,107.0,Male,Limb,Placebo,1,54.762491,13.600526,32.600526,-4.921357,licals
1,P01002,1.0,99.0,Female,Limb,Placebo,1,61.160849,20.137976,41.538765,-6.347018,licals
2,P01003,0.0,102.0,Male,Limb,Active,1,46.702259,15.571616,34.571616,-6.373852,licals


In [7]:
ril_301 = dfs[dfs['Study_id'] == '301']
ril_301 = ril_301.drop(columns=['Study_id'])

print(f"{ril_301['Expt'].value_counts().to_dict()}\n")
print(ril_301.info())
ril_301.head(3)

{'riluzole': 959}

<class 'pandas.core.frame.DataFrame'>
Index: 959 entries, 551 to 1509
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   subject_id        959 non-null    object 
 1   Event             959 non-null    float64
 2   Vital_capacity    959 non-null    float64
 3   Sex               959 non-null    object 
 4   Onset_site        959 non-null    object 
 5   Study_Arm         959 non-null    object 
 6   European          959 non-null    int64  
 7   Age               959 non-null    float64
 8   Diagnostic_Delay  959 non-null    float64
 9   Disease_Duration  959 non-null    float64
 10  TRICALS           959 non-null    float64
 11  Expt              959 non-null    object 
dtypes: float64(6), int64(1), object(5)
memory usage: 97.4+ KB
None


,subject_id,Event,Vital_capacity,Sex,Onset_site,Study_Arm,European,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Expt
551,11001,1.0,74.619023,1,1.0,ACTIVE,1,64.0,0.229961,7.161629,-1.973438,riluzole
552,11002,0.0,127.558757,1,1.0,PLACEBO,1,42.0,0.000000,19.152431,-3.891616,riluzole
553,11003,1.0,80.671485,1,1.0,ACTIVE,1,62.0,0.459921,18.232589,-2.340926,riluzole


In [8]:
proact = pd.read_csv('/Users/Apple/projects/ALS_Digital_Twins/01_DigitalTwin_2/Processed_Data_2/ALS_Proact_Data_with_tricals_risk_profile.csv')
proact.columns

proact = proact[['subject_id', 'Event', 'vc','Sex', 'Onset_site','Study_Arm', 'European', 'Age','Diag_delay',  'Disease_Duration', 'TRICALS_Risk_Profile']]
proact['Expt'] = 'proact'

proact = proact.rename(columns={'vc': 'Vital_capacity',
                                'Diag_delay': 'Diagnostic_Delay',
                                'TRICALS_Risk_Profile': 'TRICALS'})

print(proact.info())
proact.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4771 entries, 0 to 4770
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   subject_id        4771 non-null   int64  
 1   Event             4771 non-null   int64  
 2   Vital_capacity    4771 non-null   float64
 3   Sex               4771 non-null   object 
 4   Onset_site        4771 non-null   object 
 5   Study_Arm         4771 non-null   object 
 6   European          4771 non-null   float64
 7   Age               4771 non-null   float64
 8   Diagnostic_Delay  4771 non-null   float64
 9   Disease_Duration  4771 non-null   float64
 10  TRICALS           4771 non-null   float64
 11  Expt              4771 non-null   object 
dtypes: float64(6), int64(2), object(4)
memory usage: 447.4+ KB
None


,subject_id,Event,Vital_capacity,Sex,Onset_site,Study_Arm,European,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Expt
0,121,1,63.037137,Female,limb,Active,1.0,52.00,9.607096,31.849540,-4.184806,proact
1,226,1,56.228941,Male,bulbar,Active,1.0,72.00,10.320302,29.113666,-3.279029,proact
2,624,1,37.825112,Female,limb,Active,1.0,65.14,13.095926,28.748357,-3.282845,proact


In [9]:
miroli.head(3)

,subject_id,Event,Vital_capacity,Sex,Onset_site,Study_Arm,European,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Expt
0,P01001,0.0,107.0,Male,Limb,Placebo,1,54.762491,13.600526,32.600526,-4.921357,licals
1,P01002,1.0,99.0,Female,Limb,Placebo,1,61.160849,20.137976,41.538765,-6.347018,licals
2,P01003,0.0,102.0,Male,Limb,Active,1,46.702259,15.571616,34.571616,-6.373852,licals


In [10]:
ril_301.head(3)

,subject_id,Event,Vital_capacity,Sex,Onset_site,Study_Arm,European,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Expt
551,11001,1.0,74.619023,1,1.0,ACTIVE,1,64.0,0.229961,7.161629,-1.973438,riluzole
552,11002,0.0,127.558757,1,1.0,PLACEBO,1,42.0,0.000000,19.152431,-3.891616,riluzole
553,11003,1.0,80.671485,1,1.0,ACTIVE,1,62.0,0.459921,18.232589,-2.340926,riluzole


In [11]:
proact.head(3)

,subject_id,Event,Vital_capacity,Sex,Onset_site,Study_Arm,European,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Expt
0,121,1,63.037137,Female,limb,Active,1.0,52.00,9.607096,31.849540,-4.184806,proact
1,226,1,56.228941,Male,bulbar,Active,1.0,72.00,10.320302,29.113666,-3.279029,proact
2,624,1,37.825112,Female,limb,Active,1.0,65.14,13.095926,28.748357,-3.282845,proact


#### check value counts for categorical and object columns

In [12]:
# miroli dataset
check_value_counts(miroli.drop(columns=['subject_id']))

Sex: {'M': 136, 'Male': 126, 'F': 84, 'Female': 50}

Onset_site: {'Limb': 318, 'Bulbar': 77, 'Other': 1}

Study_Arm: {'Placebo': 197, 'IL2': 110, 'Active': 89}

Expt: {'mirocals': 220, 'licals': 176}



In [13]:
# riluzole dataset
ril_301 = ril_301[ril_301['Diagnostic_Delay'] >= 0]
check_value_counts(ril_301.drop(columns=['subject_id']))

Sex: {'1': 573, '0': 384}

Onset_site: {'1.0': 913, '2.0': 44}

Study_Arm: {'ACTIVE': 715, 'PLACEBO': 242}

Expt: {'riluzole': 957}



In [14]:
# proact dataset
proact = proact[proact['Onset_site'] != 'other']
proact = proact[proact['Diagnostic_Delay'] >= 0]
check_value_counts(proact.drop(columns=['subject_id']))

Sex: {'Male': 2726, 'Female': 1876}

Onset_site: {'limb': 3565, 'bulbar': 1037}

Study_Arm: {'Active': 3055, 'Placebo': 1547}

Expt: {'proact': 4602}



In [15]:
miroli[['Disease_Duration', 'Diagnostic_Delay', 'Vital_capacity']].describe()
ril_301[['Disease_Duration', 'Diagnostic_Delay', 'Vital_capacity']].describe()
proact[['Disease_Duration', 'Diagnostic_Delay', 'Vital_capacity']].describe()


,Disease_Duration,Diagnostic_Delay,Vital_capacity
count,4602.000000,4602.000000,4602.000000
mean,29.065190,10.931628,66.689094
std,10.582427,5.492224,22.020645
min,4.664915,0.000000,17.632020
25%,21.911958,7.862681,53.482888
50%,27.524474,10.614980,65.740687
75%,33.964602,13.095926,76.124868
max,146.714849,88.961892,252.255771


In [16]:
# Log-transform specified columns
miroli = log_transform_columns(miroli, ['Disease_Duration', 'Diagnostic_Delay', 'Vital_capacity'])
ril_301 = log_transform_columns(ril_301, ['Disease_Duration', 'Diagnostic_Delay', 'Vital_capacity'])
proact = log_transform_columns(proact, ['Disease_Duration', 'Diagnostic_Delay', 'Vital_capacity'])
ril_301.head()

,subject_id,Event,Vital_capacity,Sex,Onset_site,Study_Arm,European,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Expt,Disease_Duration_l,Diagnostic_Delay_l,Vital_capacity_l
551,11001,1.0,74.619023,1,1.0,ACTIVE,1,64.0,0.229961,7.161629,-1.973438,riluzole,2.099444,0.206982,4.325708
552,11002,0.0,127.558757,1,1.0,PLACEBO,1,42.0,0.000000,19.152431,-3.891616,riluzole,3.003325,0.000000,4.856386
553,11003,1.0,80.671485,1,1.0,ACTIVE,1,62.0,0.459921,18.232589,-2.340926,riluzole,2.956606,0.378382,4.402705
554,11004,0.0,100.882268,0,1.0,ACTIVE,1,65.0,0.000000,19.086728,-2.869897,riluzole,3.000059,0.000000,4.623818
555,11005,1.0,85.222830,1,1.0,ACTIVE,1,52.0,0.000000,17.444152,-3.047773,riluzole,2.914747,0.000000,4.456935


In [17]:
# Make some columns categorical
miroli = make_columns_object(miroli, ['European', 'Event'])
ril_301 = make_columns_object(ril_301, ['European', 'Event'])
proact = make_columns_object(proact, ['European', 'Event'])

# proact.info()
miroli.head(3)
# ril_301.head(3)

,subject_id,Event,Vital_capacity,Sex,Onset_site,Study_Arm,European,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Expt,Disease_Duration_l,Diagnostic_Delay_l,Vital_capacity_l
0,P01001,0.0,107.0,Male,Limb,Placebo,1,54.762491,13.600526,32.600526,-4.921357,licals,3.514542,2.681058,4.682131
1,P01002,1.0,99.0,Female,Limb,Placebo,1,61.160849,20.137976,41.538765,-6.347018,licals,3.750416,3.051071,4.605170
2,P01003,0.0,102.0,Male,Limb,Active,1,46.702259,15.571616,34.571616,-6.373852,licals,3.571548,2.807691,4.634729


In [18]:
# standardize columns
miroli = standardize_column(miroli)
ril_301 = standardize_column(ril_301)
proact = standardize_column(proact)

ril_301.head()

,subject_id,Event,Vital_capacity,Sex,Onset_site,Study_Arm,European,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Expt,Disease_Duration_l,Diagnostic_Delay_l,Vital_capacity_l,Vital_capacity_std,Age_std,Diagnostic_Delay_std,Disease_Duration_std,TRICALS_std,Disease_Duration_l_std,Diagnostic_Delay_l_std,Vital_capacity_l_std
551,11001,1.0,74.619023,1,1.0,ACTIVE,1,64.0,0.229961,7.161629,-1.973438,riluzole,2.099444,0.206982,4.325708,-0.721047,0.667003,0.424070,-1.329288,1.193454,-1.002116,0.607142,-0.680409
552,11002,0.0,127.558757,1,1.0,PLACEBO,1,42.0,0.000000,19.152431,-3.891616,riluzole,3.003325,0.000000,4.856386,2.087427,-1.337433,-0.482680,0.979464,-1.652662,0.757981,-0.530260,1.841255
553,11003,1.0,80.671485,1,1.0,ACTIVE,1,62.0,0.459921,18.232589,-2.340926,riluzole,2.956606,0.378382,4.402705,-0.399961,0.484782,1.330821,0.802354,0.648189,0.667007,1.549016,-0.314536
554,11004,0.0,100.882268,0,1.0,ACTIVE,1,65.0,0.000000,19.086728,-2.869897,riluzole,3.000059,0.000000,4.623818,0.672229,0.758114,-0.482680,0.966813,-0.136676,0.751622,-0.530260,0.736143
555,11005,1.0,85.222830,1,1.0,ACTIVE,1,52.0,0.000000,17.444152,-3.047773,riluzole,2.914747,0.000000,4.456935,-0.158510,-0.426326,-0.482680,0.650546,-0.400601,0.585497,-0.530260,-0.056847


In [19]:
# plot histograms with density
# plot_histograms_with_kde(miroli.drop(columns=['subject_id']))

In [20]:
# plot_histograms_with_kde(ril_301.drop(columns=['subject_id']))

In [21]:
# plot_histograms_with_kde(proact.drop(columns=['subject_id']))

In [22]:
check_value_counts(miroli.drop(columns=['subject_id']))


Event: {0.0: 240, 1.0: 156}

Sex: {'M': 136, 'Male': 126, 'F': 84, 'Female': 50}

Onset_site: {'Limb': 318, 'Bulbar': 77, 'Other': 1}

Study_Arm: {'Placebo': 197, 'IL2': 110, 'Active': 89}

European: {1: 392, 0: 4}

Expt: {'mirocals': 220, 'licals': 176}



In [23]:
check_value_counts(ril_301.drop(columns=['subject_id']))


Event: {0.0: 531, 1.0: 426}

Sex: {'1': 573, '0': 384}

Onset_site: {'1.0': 913, '2.0': 44}

Study_Arm: {'ACTIVE': 715, 'PLACEBO': 242}

European: {1: 936, 2: 8, 3: 6, 4: 6, 5: 1}

Expt: {'riluzole': 957}



In [24]:
check_value_counts(proact.drop(columns=['subject_id']))

Event: {1: 3366, 0: 1236}

Sex: {'Male': 2726, 'Female': 1876}

Onset_site: {'limb': 3565, 'bulbar': 1037}

Study_Arm: {'Active': 3055, 'Placebo': 1547}

European: {1.0: 4480, 0.0: 122}

Expt: {'proact': 4602}



In [25]:
# miroli dataset
mapping = {'Limb': 'Limb', 'Bulbar': 'Bulbar',
           'limb': 'Limb', 'bulbar': 'Bulbar',
            'Other': 'Limb', '1.0': 'Limb',
           '0.0': 'Bulbar', '2.0': 'Bulbar', 1.0: 'Limb', 0.0: 'Bulbar'}

miroli['Onset_site'] = miroli['Onset_site'].map(mapping)

#________
mapping1 = {'M': 'Male', 'F': 'Female', 'Male': 'Male','Female': 'Female', '1': 'Male', '0': 'Female'}

miroli['Sex'] = miroli['Sex'].map(mapping1)

#________
mapping2 = {'IL2': 'Active', 'ACTIVE': 'Active','PLACEBO': 'Placebo', 
            'Active': 'Active','Placebo': 'Placebo'}

miroli['Study_Arm'] = miroli['Study_Arm'].map(mapping2)

#________
miroli['European'] = np.where(miroli['European'] == 0, 'Non-European', 'European')

In [26]:
# riluzole dataset
mapping = {'Limb': 'Limb', 'Bulbar': 'Bulbar',
           'limb': 'Limb', 'bulbar': 'Bulbar',
            'Other': 'Limb', '1.0': 'Limb',
           '0.0': 'Bulbar', '2.0': 'Bulbar', 1.0: 'Limb', 0.0: 'Bulbar'}


ril_301['Onset_site'] = ril_301['Onset_site'].map(mapping)

#________
mapping1 = {'M': 'Male', 'F': 'Female', 'Male': 'Male','Female': 'Female', '1': 'Male', '0': 'Female'}

ril_301['Sex'] = ril_301['Sex'].map(mapping1)

#________
mapping2 = {'IL2': 'Active', 'ACTIVE': 'Active','PLACEBO': 'Placebo', 
            'Active': 'Active','Placebo': 'Placebo'}

ril_301['Study_Arm'] = ril_301['Study_Arm'].map(mapping2)

#________
ril_301['European'] = np.where(ril_301['European'] == 0, 'Non-European', 'European')

In [27]:
# proact dataset
mapping = {'Limb': 'Limb', 'Bulbar': 'Bulbar',
           'limb': 'Limb', 'bulbar': 'Bulbar',
            'Other': 'Limb', '1.0': 'Limb',
           '0.0': 'Bulbar', '2.0': 'Bulbar', 1.0: 'Limb', 0.0: 'Bulbar'}


proact['Onset_site'] = proact['Onset_site'].map(mapping)

#________
mapping1 = {'M': 'Male', 'F': 'Female', 'Male': 'Male','Female': 'Female', '1': 'Male', '0': 'Female'}

proact['Sex'] = proact['Sex'].map(mapping1)

#________
mapping2 = {'IL2': 'Active', 'ACTIVE': 'Active','PLACEBO': 'Placebo', 
            'Active': 'Active','Placebo': 'Placebo'}

proact['Study_Arm'] = proact['Study_Arm'].map(mapping2)

#________
proact['European'] = np.where(proact['European'] == 0, 'Non-European', 'European')

In [28]:
check_value_counts(miroli.drop(columns=['subject_id']))


Event: {0.0: 240, 1.0: 156}

Sex: {'Male': 262, 'Female': 134}

Onset_site: {'Limb': 319, 'Bulbar': 77}

Study_Arm: {'Active': 199, 'Placebo': 197}

European: {'European': 392, 'Non-European': 4}

Expt: {'mirocals': 220, 'licals': 176}



In [29]:
check_value_counts(ril_301.drop(columns=['subject_id']))


Event: {0.0: 531, 1.0: 426}

Sex: {'Male': 573, 'Female': 384}

Onset_site: {'Limb': 913, 'Bulbar': 44}

Study_Arm: {'Active': 715, 'Placebo': 242}

European: {'European': 957}

Expt: {'riluzole': 957}



In [30]:
check_value_counts(proact.drop(columns=['subject_id']))

Event: {1: 3366, 0: 1236}

Sex: {'Male': 2726, 'Female': 1876}

Onset_site: {'Limb': 3565, 'Bulbar': 1037}

Study_Arm: {'Active': 3055, 'Placebo': 1547}

European: {'European': 4480, 'Non-European': 122}

Expt: {'proact': 4602}



In [31]:
proact.head(3)

,subject_id,Event,Vital_capacity,Sex,Onset_site,Study_Arm,European,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Expt,Disease_Duration_l,Diagnostic_Delay_l,Vital_capacity_l,Vital_capacity_std,Age_std,Diagnostic_Delay_std,Disease_Duration_std,TRICALS_std,Disease_Duration_l_std,Diagnostic_Delay_l_std,Vital_capacity_l_std
0,121,1,63.037137,Female,Limb,Active,European,52.00,9.607096,31.849540,-4.184806,proact,3.491938,2.361523,4.159463,-0.165842,-0.801992,-0.241165,0.263111,-0.343027,0.430658,-0.035771,-0.009882
1,226,1,56.228941,Male,Bulbar,Active,European,72.00,10.320302,29.113666,-3.279029,proact,3.404979,2.426598,4.047060,-0.475016,1.288977,-0.111307,0.004581,0.513738,0.172570,0.102079,-0.351747
2,624,1,37.825112,Female,Limb,Active,European,65.14,13.095926,28.748357,-3.282845,proact,3.392774,2.645886,3.659067,-1.310769,0.571775,0.394066,-0.029939,0.510128,0.136346,0.566605,-1.531791


In [32]:
# Identify categorical columns
cat_cols = ['Onset_site', 'European', 'Sex', 'Study_Arm','Expt']

# One-hot encode
miroli_onehot = pd.get_dummies(miroli, columns=cat_cols, drop_first=True)
ril_301_onehot = pd.get_dummies(ril_301, columns=cat_cols, drop_first=True)
proact_onehot = pd.get_dummies(proact, columns=cat_cols, drop_first=True)

# df_onehot.head(2)

In [33]:
proact_onehot.columns

Index(['subject_id', 'Event', 'Vital_capacity', 'Age', 'Diagnostic_Delay',
       'Disease_Duration', 'TRICALS', 'Disease_Duration_l',
       'Diagnostic_Delay_l', 'Vital_capacity_l', 'Vital_capacity_std',
       'Age_std', 'Diagnostic_Delay_std', 'Disease_Duration_std',
       'TRICALS_std', 'Disease_Duration_l_std', 'Diagnostic_Delay_l_std',
       'Vital_capacity_l_std', 'Onset_site_Limb', 'European_Non-European',
       'Sex_Male', 'Study_Arm_Placebo'],
      dtype='object')

In [34]:
cols = ['Onset_site_Limb', 'Sex_Male', 'Study_Arm_Placebo']

# Convert one-hot encoded columns to int
for col in cols:
    miroli_onehot[col] = miroli_onehot[col].astype(int)

# Convert one-hot encoded columns to int
for col in cols:
    ril_301_onehot[col] = ril_301_onehot[col].astype(int)


# Convert one-hot encoded columns to int
for col in cols:
    proact_onehot[col] = proact_onehot[col].astype(int)

ril_301_onehot.head(3)

,subject_id,Event,Vital_capacity,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Disease_Duration_l,Diagnostic_Delay_l,Vital_capacity_l,Vital_capacity_std,Age_std,Diagnostic_Delay_std,Disease_Duration_std,TRICALS_std,Disease_Duration_l_std,Diagnostic_Delay_l_std,Vital_capacity_l_std,Onset_site_Limb,Sex_Male,Study_Arm_Placebo
551,11001,1.0,74.619023,64.0,0.229961,7.161629,-1.973438,2.099444,0.206982,4.325708,-0.721047,0.667003,0.424070,-1.329288,1.193454,-1.002116,0.607142,-0.680409,1,1,0
552,11002,0.0,127.558757,42.0,0.000000,19.152431,-3.891616,3.003325,0.000000,4.856386,2.087427,-1.337433,-0.482680,0.979464,-1.652662,0.757981,-0.530260,1.841255,1,1,1
553,11003,1.0,80.671485,62.0,0.459921,18.232589,-2.340926,2.956606,0.378382,4.402705,-0.399961,0.484782,1.330821,0.802354,0.648189,0.667007,1.549016,-0.314536,1,1,0


In [35]:
# Create interactions
miroli_onehot['Sex_onset'] = miroli_onehot['Sex_Male']*miroli_onehot['Onset_site_Limb']
miroli_onehot['Age_Sex'] = miroli_onehot['Sex_Male']*miroli_onehot['Age_std']
miroli_onehot['Age_onset'] = miroli_onehot['Age_std']*miroli_onehot['Onset_site_Limb']
miroli_onehot['Age_TRICALS'] = miroli_onehot['Age_std']*miroli_onehot['TRICALS_std']
# miroli_onehot['Trical : Placebo Arm'] = miroli_onehot['TRICALS']*miroli_onehot['Study_Arm_Placebo']

# miroli_onehot.info()
miroli_onehot.head(2)

,subject_id,Event,Vital_capacity,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Disease_Duration_l,Diagnostic_Delay_l,Vital_capacity_l,Vital_capacity_std,Age_std,Diagnostic_Delay_std,Disease_Duration_std,TRICALS_std,Disease_Duration_l_std,Diagnostic_Delay_l_std,Vital_capacity_l_std,Onset_site_Limb,European_Non-European,Sex_Male,Study_Arm_Placebo,Expt_mirocals,Sex_onset,Age_Sex,Age_onset,Age_TRICALS
0,P01001,0.0,107.0,54.762491,13.600526,32.600526,-4.921357,3.514542,2.681058,4.682131,0.787136,-0.355001,0.168722,0.329936,-0.075830,0.412636,0.363265,0.828441,1,False,1,1,False,1,-0.355001,-0.355001,0.026920
1,P01002,1.0,99.0,61.160849,20.137976,41.538765,-6.347018,3.750416,3.051071,4.605170,0.316991,0.255274,1.271281,1.373002,-1.168542,1.130829,1.103061,0.398812,1,False,0,1,False,0,0.000000,0.255274,-0.298299


In [36]:
# Create interactions
ril_301_onehot['Sex_onset'] = ril_301_onehot['Sex_Male']*ril_301_onehot['Onset_site_Limb']
ril_301_onehot['Age_Sex'] = ril_301_onehot['Sex_Male']*ril_301_onehot['Age_std']
ril_301_onehot['Age_onset'] = ril_301_onehot['Age_std']*ril_301_onehot['Onset_site_Limb']
ril_301_onehot['Age_TRICALS'] = ril_301_onehot['Age_std']*ril_301_onehot['TRICALS_std']
# ril_301_onehot['Trical : Placebo Arm'] = ril_301_onehot['TRICALS']*ril_301_onehot['Study_Arm_Placebo']

# ril_301_onehot.info()
ril_301_onehot.head(2)

,subject_id,Event,Vital_capacity,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Disease_Duration_l,Diagnostic_Delay_l,Vital_capacity_l,Vital_capacity_std,Age_std,Diagnostic_Delay_std,Disease_Duration_std,TRICALS_std,Disease_Duration_l_std,Diagnostic_Delay_l_std,Vital_capacity_l_std,Onset_site_Limb,Sex_Male,Study_Arm_Placebo,Sex_onset,Age_Sex,Age_onset,Age_TRICALS
551,11001,1.0,74.619023,64.0,0.229961,7.161629,-1.973438,2.099444,0.206982,4.325708,-0.721047,0.667003,0.42407,-1.329288,1.193454,-1.002116,0.607142,-0.680409,1,1,0,1,0.667003,0.667003,0.796037
552,11002,0.0,127.558757,42.0,0.000000,19.152431,-3.891616,3.003325,0.000000,4.856386,2.087427,-1.337433,-0.48268,0.979464,-1.652662,0.757981,-0.530260,1.841255,1,1,1,1,-1.337433,-1.337433,2.210326


In [37]:
# Create interactions
proact_onehot['Sex_onset'] = proact_onehot['Sex_Male']*proact_onehot['Onset_site_Limb']
proact_onehot['Age_Sex'] = proact_onehot['Sex_Male']*proact_onehot['Age_std']
proact_onehot['Age_onset'] = proact_onehot['Age_std']*proact_onehot['Onset_site_Limb']
proact_onehot['Age_TRICALS'] = proact_onehot['Age_std']*proact_onehot['TRICALS_std']
# proact_onehot['Trical : Placebo Arm'] = proact_onehot['TRICALS']*proact_onehot['Study_Arm_Placebo']

# miroli_onehot.info()
proact_onehot.head(2)

,subject_id,Event,Vital_capacity,Age,Diagnostic_Delay,Disease_Duration,TRICALS,Disease_Duration_l,Diagnostic_Delay_l,Vital_capacity_l,Vital_capacity_std,Age_std,Diagnostic_Delay_std,Disease_Duration_std,TRICALS_std,Disease_Duration_l_std,Diagnostic_Delay_l_std,Vital_capacity_l_std,Onset_site_Limb,European_Non-European,Sex_Male,Study_Arm_Placebo,Sex_onset,Age_Sex,Age_onset,Age_TRICALS
0,121,1,63.037137,52.0,9.607096,31.849540,-4.184806,3.491938,2.361523,4.159463,-0.165842,-0.801992,-0.241165,0.263111,-0.343027,0.430658,-0.035771,-0.009882,1,False,0,0,0,-0.000000,-0.801992,0.275105
1,226,1,56.228941,72.0,10.320302,29.113666,-3.279029,3.404979,2.426598,4.047060,-0.475016,1.288977,-0.111307,0.004581,0.513738,0.172570,0.102079,-0.351747,0,False,1,0,0,1.288977,0.000000,0.662197


In [38]:
miroli0 = miroli_onehot.copy()
ril_3010 = ril_301_onehot.copy()
proact0 = proact_onehot.copy()


In [39]:
proact0.columns

Index(['subject_id', 'Event', 'Vital_capacity', 'Age', 'Diagnostic_Delay',
       'Disease_Duration', 'TRICALS', 'Disease_Duration_l',
       'Diagnostic_Delay_l', 'Vital_capacity_l', 'Vital_capacity_std',
       'Age_std', 'Diagnostic_Delay_std', 'Disease_Duration_std',
       'TRICALS_std', 'Disease_Duration_l_std', 'Diagnostic_Delay_l_std',
       'Vital_capacity_l_std', 'Onset_site_Limb', 'European_Non-European',
       'Sex_Male', 'Study_Arm_Placebo', 'Sex_onset', 'Age_Sex', 'Age_onset',
       'Age_TRICALS'],
      dtype='object')

In [40]:
miroli0 = miroli0[['subject_id', 'Event', 'Disease_Duration', 'Study_Arm_Placebo',
       'Age_std', 'TRICALS_std', 'Diagnostic_Delay_l_std', 'Vital_capacity_l_std', 'Onset_site_Limb', 
        'Sex_Male','Sex_onset', 'Age_Sex', 'Age_onset', 'Age_TRICALS']]

miroli0 = miroli0.rename(columns={'Age_std': 'Age',
                        'Vital_capacity_l_std': 'Vital_capacity',
                        'Diagnostic_Delay_l_std': 'Diagnostic_Delay',
                        'TRICALS_std': 'TRICALS',
                        'European_Non-European': 'European_Yes',
                        'Onset_site_Limb': 'Onset_Limb'
                        })

miroli0.columns

Index(['subject_id', 'Event', 'Disease_Duration', 'Study_Arm_Placebo', 'Age',
       'TRICALS', 'Diagnostic_Delay', 'Vital_capacity', 'Onset_Limb',
       'Sex_Male', 'Sex_onset', 'Age_Sex', 'Age_onset', 'Age_TRICALS'],
      dtype='object')

In [41]:
ril_3010 = ril_3010[['subject_id', 'Event', 'Disease_Duration', 'Study_Arm_Placebo',
       'Age_std', 'TRICALS_std', 'Diagnostic_Delay_l_std', 'Vital_capacity_l_std', 'Onset_site_Limb', 
        'Sex_Male','Sex_onset', 'Age_Sex', 'Age_onset', 'Age_TRICALS']]

ril_3010 = ril_3010.rename(columns={'Age_std': 'Age',
                        'Vital_capacity_l_std': 'Vital_capacity',
                        'Diagnostic_Delay_l_std': 'Diagnostic_Delay',
                        'TRICALS_std': 'TRICALS',
                        'European_Non-European': 'European_Yes',
                        'Onset_site_Limb': 'Onset_Limb'
                        })

ril_3010.columns

Index(['subject_id', 'Event', 'Disease_Duration', 'Study_Arm_Placebo', 'Age',
       'TRICALS', 'Diagnostic_Delay', 'Vital_capacity', 'Onset_Limb',
       'Sex_Male', 'Sex_onset', 'Age_Sex', 'Age_onset', 'Age_TRICALS'],
      dtype='object')

In [42]:
proact0 = proact0[['subject_id', 'Event', 'Disease_Duration', 'Study_Arm_Placebo',
       'Age_std', 'TRICALS_std', 'Diagnostic_Delay_l_std', 'Vital_capacity_l_std', 'Onset_site_Limb', 
        'Sex_Male','Sex_onset', 'Age_Sex', 'Age_onset', 'Age_TRICALS']]

proact0 = proact0.rename(columns={'Age_std': 'Age',
                        'Vital_capacity_l_std': 'Vital_capacity',
                        'Diagnostic_Delay_l_std': 'Diagnostic_Delay',
                        'TRICALS_std': 'TRICALS',
                        'European_Non-European': 'European_Yes',
                        'Onset_site_Limb': 'Onset_Limb'
                        })

proact0.columns

Index(['subject_id', 'Event', 'Disease_Duration', 'Study_Arm_Placebo', 'Age',
       'TRICALS', 'Diagnostic_Delay', 'Vital_capacity', 'Onset_Limb',
       'Sex_Male', 'Sex_onset', 'Age_Sex', 'Age_onset', 'Age_TRICALS'],
      dtype='object')

In [43]:
miroli0.head(3)

,subject_id,Event,Disease_Duration,Study_Arm_Placebo,Age,TRICALS,Diagnostic_Delay,Vital_capacity,Onset_Limb,Sex_Male,Sex_onset,Age_Sex,Age_onset,Age_TRICALS
0,P01001,0.0,32.600526,1,-0.355001,-0.075830,0.363265,0.828441,1,1,1,-0.355001,-0.355001,0.026920
1,P01002,1.0,41.538765,1,0.255274,-1.168542,1.103061,0.398812,1,0,0,0.000000,0.255274,-0.298299
2,P01003,0.0,34.571616,0,-1.123785,-1.189109,0.616453,0.563821,1,1,1,-1.123785,-1.123785,1.336303


In [44]:
ril_3010.head(3)

,subject_id,Event,Disease_Duration,Study_Arm_Placebo,Age,TRICALS,Diagnostic_Delay,Vital_capacity,Onset_Limb,Sex_Male,Sex_onset,Age_Sex,Age_onset,Age_TRICALS
551,11001,1.0,7.161629,0,0.667003,1.193454,0.607142,-0.680409,1,1,1,0.667003,0.667003,0.796037
552,11002,0.0,19.152431,1,-1.337433,-1.652662,-0.530260,1.841255,1,1,1,-1.337433,-1.337433,2.210326
553,11003,1.0,18.232589,0,0.484782,0.648189,1.549016,-0.314536,1,1,1,0.484782,0.484782,0.314230


In [45]:
miroli0 = make_columns_int(miroli0, ["Event"])
ril_3010 = make_columns_int(ril_3010, ["Event"])
proact0 = make_columns_int(proact0, ["Event"])

In [46]:
# proact0.to_csv('/Users/Apple/projects/ALS_Digital_Twins/All_processed_data/Results/proact0_processed_data_for_fp_model_21-01-2026.csv', index=False)
# ril_3010.to_csv('/Users/Apple/projects/ALS_Digital_Twins/All_processed_data/Results/ril_3010_processed_data_for_fp_model_21-01-2026.csv', index=False)
# miroli0.to_csv('/Users/Apple/projects/ALS_Digital_Twins/All_processed_data/Results/miroli0_processed_data_for_fp_model_21-01-2026.csv', index=False)

#### CV Backward stepwise selection based on AIC and spline df selection
<!-- ##### investigate the code further -->

In [47]:
# Candidate covariates (modify for your dataset)
covariates = ['Study_Arm_Placebo', 'Age', 'TRICALS', 'Diagnostic_Delay', 'Study_Arm_Placebo',
              'Vital_capacity', 'Onset_Limb', 'Sex_Male', 'Sex_onset',
              'Age_Sex', 'Age_onset', 'Age_TRICALS']

df_candidates = [1, 2, 3, 4, 5]

In [48]:
%%writefile going_modular/test_harrels_fp_model_with_cross_validation.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Optional, Tuple, Dict
from lifelines import KaplanMeierFitter, CoxPHFitter, NelsonAalenFitter
from sklearn.model_selection import KFold
from scipy.stats import gaussian_kde, norm
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import default_converter
from rpy2.robjects import pandas2ri, conversion
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import Formula
from rpy2.robjects.vectors import FloatVector, IntVector, ListVector, Vector, StrVector
from rpy2.robjects import DataFrame, IntVector

rstpm2 = importr("rstpm2")
survival = importr("survival")
ggplot2 = importr("ggplot2")
graphics = importr("graphics")
stats = importr("stats")
lmtest = importr("lmtest")


def test_harrels_fp_model_with_cross_validation(df_python, vars_list, spline_df, n_splits=5):
    """
    Test a survival model using cross-validation.
    Returns average AIC and average C-index.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    aics = []
    c_indices = []

    rhs = " + ".join(vars_list) if vars_list else "1"
    formula_str = f"Surv(Disease_Duration, Event==1) ~ {rhs}"
    formula = Formula(formula_str)
    

    for train_idx, test_idx in kf.split(df_python):
        train_df = df_python.iloc[train_idx]
        test_df = df_python.iloc[test_idx]

        with localconverter(default_converter + pandas2ri.converter):
            r_train = pandas2ri.py2rpy(train_df)
            r_test = pandas2ri.py2rpy(test_df)

        try:
            # Fit model on training data
            model = rstpm2.stpm2(formula, data=r_train, df=spline_df)
            
            # Calculate AIC on training data
            aic = stats.AIC(model)[0]
            aics.append(aic)
            
            # Calculate C-index on test data
            # Get predictions for test set
            r_predict = ro.r['predict']
            predictions = r_predict(model, newdata=r_test, type="link")
            # print(ro.r("is.null")(predictions))
            # print(ro.r("length")(predictions))
            # print(ro.r("class")(predictions))

            # Calculate concordance using survival package
            r_test = ro.r("data.frame")(r_test)
            # print(r_test.names)        # Column names
            # print(r["nrow"](r_test))

            r_test = ro.r("transform")(r_test, predicted=predictions)
            # print(r_test.names)
            # print(r["tail"](r_test))

            # print(len(predictions), r["nrow"](r_test)[0])

            # r_test.rx2['predicted'] = predictions
            c_index = survival.concordance(
                ro.Formula(f"Surv(Disease_Duration, Event==1) ~ predicted"),
                data=r_test
            )[0]  # First element is the C-index
            
            c_indices.append(c_index)
            
        except Exception as e:
            print(f"Model fitting failed in fold: {e}")
            aics.append(np.inf)
            c_indices.append(0)  # Worst possible C-index

    return np.mean(aics), np.mean(c_indices)

Overwriting going_modular/test_harrels_fp_model_with_cross_validation.py


In [49]:
%%writefile going_modular/find_best_harrels_fp_model_by_backward_elimination.py
import numpy as np
from typing import List, Tuple
from going_modular.test_harrels_fp_model_with_cross_validation import test_harrels_fp_model_with_cross_validation

def find_best_harrels_fp_model_by_backward_elimination(train_df, covariates, df_candidates, n_splits=5):
    """
    Find the best model by testing different variable combinations.
    Uses both AIC (lower is better) and C-index (higher is better).
    """
    current_vars = covariates.copy()
    best_aic = np.inf
    best_c_index = 0
    best_vars = current_vars.copy()
    best_df = None

    while len(current_vars) > 0:
        candidates = []

        # Test current model with different spline df values
        for df_spline in df_candidates:
            aic, c_idx = test_harrels_fp_model_with_cross_validation(train_df, current_vars, df_spline, n_splits)
            candidates.append((aic, c_idx, current_vars.copy(), df_spline))

        # Test removing each variable
        for var in current_vars:
            test_vars = [v for v in current_vars if v != var]
            if len(test_vars) == 0:
                continue
            for df_spline in df_candidates:
                aic, c_idx = test_harrels_fp_model_with_cross_validation(train_df, test_vars, df_spline, n_splits)
                candidates.append((aic, c_idx, test_vars.copy(), df_spline))

        # Find best candidate (minimum AIC)
        # best_candidate = min(candidates, key=lambda x: x[0])
      
        best_candidate = max(candidates, key=lambda x: x[1])  # x[1] is C-index
        # best_candidate = max(candidates, key=lambda x: (x[1], -x[0]))  # Maximize C-index, minimize AIC
        cand_aic, cand_c_idx, cand_vars, cand_df = best_candidate

        # Stop if no improvement
        # if cand_aic >= best_aic:
        #     break
        if cand_c_idx <= best_c_index:  # Stop if C-index doesn't improve
            break

        # Update best model
        best_aic = cand_aic
        best_c_index = cand_c_idx
        best_vars = cand_vars.copy()
        best_df = cand_df
        current_vars = cand_vars.copy()

    print("\nSelected model by Harrel's C-Index:")
    print("Variables:", best_vars)
    print("Spline df:", best_df)
    print("CV-AIC:", best_aic)
    print("CV C-index:", best_c_index)
    
    return best_vars, best_df, best_aic, best_c_index

Overwriting going_modular/find_best_harrels_fp_model_by_backward_elimination.py


In [50]:
# Perform flexible parametric model cross-validation with AIC
model_0 = find_best_harrels_fp_model_by_backward_elimination(train_df=miroli0, covariates=covariates, df_candidates=df_candidates, n_splits=5)
model_0            

NameError: name 'find_best_harrels_fp_model_by_backward_elimination' is not defined

## **Bootstrapping, calibration slope, regression coefficients, CI.**

In [90]:
# define a small helper to extract C-index
def r_cindex(concordance_obj):
    # extract the concordance slot, which is an R FloatVector
    concord_vec = ro.r('function(x) x$concordance')(concordance_obj)
    return float(concord_vec[0])  # <- index 0 extracts the numeric



def test_harrels_fp_model_with_bootstrap_calibration(df_python, vars_list, spline_df, n_boot=100):
    """
    Full bootstrap internal validation for Harrell's FP model.
    Returns:
    - optimism-corrected AIC
    - optimism-corrected C-index
    - optimism-corrected calibration slope
    - 95% CI for C-index
    - 95% CI for slope
    - shrinkage-corrected regression coefficients
    - 95% CI for coefficients
    """
    rhs = " + ".join(vars_list) if vars_list else "1"
    formula_str = f"Surv(Disease_Duration, Event==1) ~ {rhs}"
    formula = Formula(formula_str)

    df_python = df_python.reset_index(drop=True)

    with localconverter(default_converter + pandas2ri.converter):
        r_df = pandas2ri.py2rpy(df_python)

    r_predict = ro.r['predict']
    r_coef = ro.r['coef']

    # Fit model on full dataset (apparent performance)
    model_full = rstpm2.stpm2(formula, data=r_df, df=spline_df)
    aic_full = stats.AIC(model_full)[0]
    predictions_full = r_predict(model_full, newdata=r_df, type="link")
    r_df_full = ro.r("transform")(r_df, predicted=predictions_full)

    # Apparent C-index
    c_index_full = r_cindex(survival.concordance(
        ro.Formula("Surv(Disease_Duration, Event==1) ~ predicted"),
        data=r_df_full
    ))

    # Apparent calibration slope
    cal_model_full = rstpm2.stpm2(
        Formula("Surv(Disease_Duration, Event==1) ~ predicted"),
        data=r_df_full, df=1
    )
    slope_full = float(np.array(r_coef(cal_model_full))[0])

    # Apparent regression coefficients
    # beta_apparent = dict(zip(vars_list, r_coef(model_full)))
    beta_apparent = {
        var: float(np.array(r_coef(model_full))[i])
        for i, var in enumerate(vars_list)
    }


    # Bootstrap storage
    cindex_boot_diff = []
    slope_boot_diff = []
    coeff_boot_diff = {var: [] for var in vars_list}
    cindex_out_all = []
    slope_out_all = []
    coeff_out_all = {var: [] for var in vars_list}

    for i in range(n_boot):
        sample_idx = np.random.choice(df_python.index, size=len(df_python), replace=True)
        boot_df = df_python.loc[sample_idx].reset_index(drop=True)

        with localconverter(default_converter + pandas2ri.converter):
            r_boot = pandas2ri.py2rpy(boot_df)
        try:
            # Fit bootstrap model
            model_boot = rstpm2.stpm2(formula, data=r_boot, df=spline_df)

            # Predictions on original data (out-of-bootstrap)
            predictions_out = r_predict(model_boot, newdata=r_df, type="link")
            r_df_out = ro.r("transform")(r_df, predicted=predictions_out)

            # C-index
            c_index_boot = r_cindex(survival.concordance(
                ro.Formula("Surv(Disease_Duration, Event==1) ~ predicted"),
                data=ro.r("transform")(r_boot, predicted=r_predict(model_boot, newdata=r_boot, type="link"))
            ))

            c_index_out = r_cindex(survival.concordance(
                ro.Formula("Surv(Disease_Duration, Event==1) ~ predicted"),
                data=r_df_out
            ))

            cindex_boot_diff.append(c_index_boot - c_index_out)
            cindex_out_all.append(c_index_out)

            # Calibration slope
            cal_model_boot = rstpm2.stpm2(
                Formula("Surv(Disease_Duration, Event==1) ~ predicted"),
                data=ro.r("transform")(r_boot, predicted=r_predict(model_boot, newdata=r_boot, type="link")),
                df=1
            )
            slope_boot = float(np.array(r_coef(cal_model_boot))[0])

            cal_model_out = rstpm2.stpm2(
                Formula("Surv(Disease_Duration, Event==1) ~ predicted"),
                data=r_df_out,
                df=1
            )
            slope_out = float(np.array(r_coef(cal_model_out))[0])
            slope_boot_diff.append(slope_boot - slope_out)
            slope_out_all.append(slope_out)

            # Regression coefficients
            # beta_boot = dict(zip(vars_list, r_coef(model_boot)))
            beta_boot = {
                var: float(np.array(r_coef(model_boot))[i])
                for i, var in enumerate(vars_list)
            }

            for var in vars_list:
                coeff_boot_diff[var].append(beta_boot[var] - beta_apparent[var])
                coeff_out_all[var].append(beta_apparent[var] - (beta_boot[var] - beta_apparent[var]))

        except Exception as e:
            print(f"Bootstrap {i} failed: {e}")
            cindex_boot_diff.append(0)
            slope_boot_diff.append(0)
            for var in vars_list:
                coeff_boot_diff[var].append(0)
                coeff_out_all[var].append(beta_apparent[var])
            cindex_out_all.append(c_index_full)
            slope_out_all.append(slope_full)

    # Corrected metrics
    corrected_aic = aic_full  # leave as apparent for now; AIC optimism can be complex
    corrected_c_index = c_index_full - np.mean(cindex_boot_diff)
    corrected_slope = slope_full - np.mean(slope_boot_diff)
    beta_corrected = {var: beta_apparent[var] - np.mean(coeff_boot_diff[var]) for var in vars_list}

    # 95% CI
    cindex_ci = np.percentile(cindex_out_all, [2.5, 97.5])
    slope_ci = np.percentile(slope_out_all, [2.5, 97.5])
    beta_ci = {var: np.percentile(coeff_out_all[var], [2.5, 97.5]) for var in vars_list}

    return beta_corrected, beta_ci, corrected_c_index, corrected_slope, cindex_ci, slope_ci, corrected_aic


In [117]:
from collections import Counter

def find_best_harrels_fp_model_with_bootstrap_elimination(train_df, covariates, df_candidates, n_boot=100, slope_penalty=0.05, random_state=None):
    """
    Backward elimination with:
    - bootstrap-corrected C-index, calibration slope, AIC
    - shrinkage-corrected regression coefficients
    - 95% confidence intervals for C-index, slope, and coefficients

    Parameters:
    - train_df: pandas DataFrame with training data
    - covariates: list of initial covariates
    - df_candidates: list of spline df values to test
    - n_boot: number of bootstrap samples
    - slope_penalty: weight to penalize calibration slope deviation from 1

    Returns:
    - best_vars: selected variables
    - best_df: selected spline df
    - best_aic: bootstrap-corrected AIC
    - best_c_index: bootstrap-corrected C-index
    - best_slope: bootstrap-corrected calibration slope
    - best_cindex_ci: 95% CI for C-index
    - best_slope_ci: 95% CI for slope
    - beta_corrected: shrinkage-corrected regression coefficients
    - beta_ci: 95% CI for regression coefficients
    """

    if random_state is not None:
        np.random.seed(random_state)

    selection_counts = Counter()

    current_vars = covariates.copy()
    best_aic = np.inf
    best_c_index = 0
    best_slope = 0
    best_vars = current_vars.copy()
    best_df = None
    best_cindex_ci = None
    best_slope_ci = None
    best_beta_corrected = None
    best_beta_ci = None

    while len(current_vars) > 0:
        candidates = []

        # Test current model with different spline df values
        for df_spline in df_candidates:
            beta_corrected, beta_ci, c_idx, slope, cindex_ci, slope_ci, aic = \
                test_harrels_fp_model_with_bootstrap_calibration(
                    train_df, current_vars, df_spline, n_boot=n_boot
                )
            candidates.append((aic, c_idx, slope, cindex_ci, slope_ci, beta_corrected, beta_ci, current_vars.copy(), df_spline))

        # Test removing each variable
        for var in current_vars:
            test_vars = [v for v in current_vars if v != var]
            if len(test_vars) == 0:
                continue
            for df_spline in df_candidates:
                beta_corrected, beta_ci, c_idx, slope, cindex_ci, slope_ci, aic = \
                    test_harrels_fp_model_with_bootstrap_calibration(
                        train_df, test_vars, df_spline, n_boot=n_boot
                    )
                candidates.append((aic, c_idx, slope, cindex_ci, slope_ci, beta_corrected, beta_ci, test_vars.copy(), df_spline))

        # Penalized score: prioritize C-index, penalize slope deviation
        def candidate_score(x):
            _, c_idx, slope, _, _, _, _, _, _ = x
            return c_idx - slope_penalty * abs(slope - 1)

        best_candidate = max(candidates, key=candidate_score)
        cand_aic, cand_c_idx, cand_slope, cand_cindex_ci, cand_slope_ci, cand_beta_corrected, cand_beta_ci, cand_vars, cand_df = best_candidate

        selection_counts.update(cand_vars)

        # Stop if no improvement
        if candidate_score(best_candidate) <= best_c_index - slope_penalty * abs(best_slope - 1):
            break

        # Update best model
        best_aic = cand_aic
        best_c_index = cand_c_idx
        best_slope = cand_slope
        best_cindex_ci = cand_cindex_ci
        best_slope_ci = cand_slope_ci
        best_beta_corrected = cand_beta_corrected
        best_beta_ci = cand_beta_ci
        best_vars = cand_vars.copy()
        best_df = cand_df
        current_vars = cand_vars.copy()

    print("\nSelected model with full bootstrap internal validation:")
    print("Variables:", best_vars)
    print("Spline df:", best_df)
    print("Bootstrap-corrected AIC:", best_aic)
    print("Bootstrap-corrected C-index:", best_c_index, "95% CI:", best_cindex_ci)
    print("Bootstrap-corrected calibration slope:", best_slope, "95% CI:", best_slope_ci)
    print("Shrinkage-corrected regression coefficients:")
    for var in best_vars:
        print(f"{var}: {best_beta_corrected[var]} 95% CI: {best_beta_ci[var]}")

    print("\nVariable selection frequencies:")
    for var in covariates:
        print(f"{var}: {selection_counts[var]}")

    return (best_vars, best_df, best_aic, best_c_index, best_slope,
            best_cindex_ci, best_slope_ci, best_beta_corrected, best_beta_ci, selection_counts)


In [124]:
# Define your initial covariates and spline degrees of freedom
covariates = ['Study_Arm_Placebo', 'Age', 'TRICALS', 'Diagnostic_Delay', 'Study_Arm_Placebo',
              'Vital_capacity', 'Onset_Limb', 'Sex_Male', 'Sex_onset']

df_candidates = [1, 2, 3]  


# Set number of bootstrap samples
n_boot = 10  # can increase for more stable estimates


# Run backward elimination with full bootstrap validation
(best_vars, best_df, best_aic, best_c_index, best_slope,
 best_cindex_ci, best_slope_ci, beta_corrected, beta_ci, selection_counts) = \
    find_best_harrels_fp_model_with_bootstrap_elimination(train_df=miroli0,
                                                        covariates=covariates,
                                                        df_candidates=df_candidates,
                                                        n_boot=n_boot,
                                                        slope_penalty=0.05,
                                                        random_state=42
                                                    )

# # Print results neatly
# print("\n===== Selected Model Summary =====")
# print(f"Variables selected: {best_vars}")
# print(f"Spline degrees of freedom: {best_df}")
# print(f"Bootstrap-corrected AIC: {best_aic:.2f}")
# print(f"Bootstrap-corrected C-index: {float(best_c_index):.3f} (95% CI: {float(best_cindex_ci[0]):.3f} - {float(best_cindex_ci[1]):.3f})")
# print(f"Bootstrap-corrected calibration slope: {best_slope:.3f} (95% CI: {best_slope_ci[0]:.3f} - {best_slope_ci[1]:.3f})")
# print("Shrinkage-corrected regression coefficients:")
# for var in best_vars:
#     print(f"  {var}: {beta_corrected[var]:.4f} (95% CI: {beta_ci[var][0]:.4f} - {beta_ci[var][1]:.4f})")



Selected model with full bootstrap internal validation:
Variables: ['Study_Arm_Placebo', 'Age', 'Diagnostic_Delay', 'Study_Arm_Placebo', 'Vital_capacity', 'Sex_Male', 'Sex_onset']
Spline df: 1
Bootstrap-corrected AIC: 1470.8041272284695
Bootstrap-corrected C-index: 0.7679792411362816 95% CI: [0.67429971 0.79156918]
Bootstrap-corrected calibration slope: -6.7779526739510745 95% CI: [-7.49669234 -5.29135953]
Shrinkage-corrected regression coefficients:
Study_Arm_Placebo: -0.5867848605101939 95% CI: [-0.6781981  -0.50850526]
Age: 0.08678315101863486 95% CI: [-0.29727459  0.45615607]
Diagnostic_Delay: 0.3827250097702914 95% CI: [0.27367724 0.47396856]
Study_Arm_Placebo: -0.5867848605101939 95% CI: [-0.6781981  -0.50850526]
Vital_capacity: -0.458953244865222 95% CI: [-0.56458788 -0.31327495]
Sex_Male: -0.016148393367901026 95% CI: [-0.45284783  0.67378044]
Sex_onset: -0.354989128864033 95% CI: [-1.0606028   0.23753179]

Variable selection frequencies:
Study_Arm_Placebo: 6
Age: 3
TRICALS: 0

In [123]:
coef_table = pd.DataFrame({
    "Variable": best_vars,
    "Coefficient": [beta_corrected[v] for v in best_vars],
    "CI_lower": [beta_ci[v][0] for v in best_vars],
    "CI_upper": [beta_ci[v][1] for v in best_vars],
    "Selection_count": [selection_counts[v] for v in best_vars]
})

coef_table.round(4)

,Variable,Coefficient,CI_lower,CI_upper,Selection_count
0,Study_Arm_Placebo,-0.5868,-0.6782,-0.5085,6
1,Age,0.0868,-0.2973,0.4562,3
2,Diagnostic_Delay,0.3827,0.2737,0.4740,3
3,Study_Arm_Placebo,-0.5868,-0.6782,-0.5085,6
4,Vital_capacity,-0.4590,-0.5646,-0.3133,3
5,Sex_Male,-0.0161,-0.4528,0.6738,2
6,Sex_onset,-0.3550,-1.0606,0.2375,3


In [127]:
performance_table = pd.DataFrame({
    "Metric": [ "C-index","Calibration slope", "AIC","Spline degrees of freedom"],
    "Estimate": [ best_c_index,best_slope, best_aic, best_df],
    "CI_lower": [ best_cindex_ci[0],best_slope_ci[0], None, None],
    "CI_upper": [ best_cindex_ci[1],best_slope_ci[1], None, None]
})

performance_table.round(4)

,Metric,Estimate,CI_lower,CI_upper
0,C-index,0.7680,0.6743,0.7916
1,Calibration slope,-6.7780,-7.4967,-5.2914
2,AIC,1470.8041,NaN,NaN
3,Spline degrees of freedom,1.0000,NaN,NaN


In [126]:
stability_table = (
    pd.DataFrame.from_dict(selection_counts, orient="index", columns=["Selected_times"])
      .reset_index()
      .rename(columns={"index": "Variable"})
      .sort_values("Selected_times", ascending=False)
)

stability_table


,Variable,Selected_times
0,Study_Arm_Placebo,6
1,Age,3
2,Diagnostic_Delay,3
3,Vital_capacity,3
6,Sex_onset,3
5,Sex_Male,2
4,Onset_Limb,1


In [135]:
rhs = " + ".join(best_vars)
formula = Formula(f"Surv(Disease_Duration, Event==1) ~ {rhs}")

with localconverter(default_converter + pandas2ri.converter):
    r_df = pandas2ri.py2rpy(miroli0)

final_model = rstpm2.stpm2(
    formula,
    data=r_df,
    df=best_df
)
print(ro.r.summary(final_model))


Maximum likelihood estimation

Call:
(function (formula, data, weights = NULL, subset = NULL, coxph.strata = NULL, 
    ...) 
{
    m <- match.call()
    m[[1L]] <- quote(gsm)
    m$penalised <- FALSE
    out <- eval(m, data, parent.frame())
    out@Call <- match.call()
    out
})(formula = Surv(Disease_Duration, Event == 1) ~ Study_Arm_Placebo + 
    Age + Diagnostic_Delay + Study_Arm_Placebo + Vital_capacity + 
    Sex_Male + Sex_onset, data = list(subject_id = c("P01001", 
"P01002", "P01003", "P01004", "P01005", "P01006", "P01007", "P01009", 
"P01010", "P01011", "P01012", "P01013", "P01014", "P01015", "P01017", 
"P01018", "P01019", "P01020", "P01022", "P01023", "P01024", "P01025", 
"P02001", "P02002", "P02003", "P02004", "P02005", "P02006", "P02007", 
"P02008", "P02009", "P02010", "P02011", "P02013", "P02014", "P02016", 
"P02017", "P02018", "P02019", "P02020", "P03001", "P03002", "P03003", 
"P03004", "P03005", "P03006", "P03007", "P03008", "P03009", "P03011", 
"P03012", "P03013", "P

In [136]:
r_coef = ro.r['coef']
beta_original = np.array(r_coef(final_model))
beta_original.round(4)

array([-4.7573e+00,  4.3700e-02,  3.7420e-01, -6.0980e-01, -4.5660e-01,
        4.5000e-03, -3.9600e-01,  6.3082e+00])

In [137]:
shrinkage_factors = np.array([
    beta_corrected[var] / beta_original[i]
    for i, var in enumerate(best_vars)
])
shrinkage_factors.round(4)

array([ 0.1233,  1.9859,  1.0227,  0.9623,  1.0051, -3.6113,  0.8965])

In [ ]:
# Get the linear predictor
lp = ro.r['predict'](
    final_model,
    newdata=r_df,
    type="link"
)
lp

-1.735359,-0.659622,-1.928122,...,-2.051002,-0.232696,0.507704


In [143]:
# Apply shrinkage to the linear predictor
lp_shrunk = np.array(lp) * np.mean(shrinkage_factors)
# lp_shrunk

In [147]:
# Predict survival at a specific time
t0 = 12  # time in same units as Disease_Duration

surv_probs = ro.r['predict'](
    final_model,
    newdata=r_df,
    type="surv",
    time=t0
)

In [148]:
new_patients = miroli0.iloc[:5].copy()

with localconverter(default_converter + pandas2ri.converter):
    r_new = pandas2ri.py2rpy(new_patients)

lp_new = ro.r['predict'](
    final_model,
    newdata=r_new,
    type="link"
)

lp_new_shrunk = np.array(lp_new) * np.mean(shrinkage_factors)

surv_new = ro.r['predict'](
    final_model,
    newdata=r_new,
    type="surv",
    time=t0
)


### *Fit a flexible parametric survival model (baseline model) -  The baseline log-cumulative-hazard* 

In [100]:
with conversion.localconverter(pandas2ri.converter):
    r_miroli0 = pandas2ri.py2rpy(miroli0)

with conversion.localconverter(pandas2ri.converter):
    r_ril_3010 = pandas2ri.py2rpy(ril_3010)

with conversion.localconverter(pandas2ri.converter):
    r_proact0 = pandas2ri.py2rpy(proact0)

In [101]:
# Define the formula
formula = Formula('Surv(Disease_Duration, Event==1) ~ Age + TRICALS + Diagnostic_Delay + Vital_capacity + Onset_Limb + Sex_Male + Sex_onset')

# Fit the flexible parametric model
# df = degrees of freedom for spline, adjust as needed
fp_model = rstpm2.stpm2(formula, data=r_miroli0, df=0)
# print(ro.r.summary(fp_model))

In [ ]:
# # Define the formula
# formula = Formula('Surv(Disease_Duration, Event==1) ~ Age  + Sex_Male + Onset_Limb + TRICALS + Diagnostic_Delay + Study_Arm_Placebo')

# # Fit the flexible parametric model
# # df = degrees of freedom for spline, adjust as needed
# fp_model = rstpm2.stpm2(formula, data=r_train_df, df=0)
# # print(ro.r.summary(fp_model))

In [102]:
df_coef = extract_coefficients_from_fp_model_log_haz_scale(fp_model)
df_coef = round(df_coef, 4)
df_coef

Available slots: [1] "frailty" "theta"   "wald"    "args"    "call"    "coef"    "m2logL" 



,Variable,Estimate,Std. Error,z value,Pr(z),LHR,CI_lower,CI_upper,Significant
0,Age,-0.4663,0.1128,-4.1332,0.0000,-0.4663,-0.6874,-0.2451,True
1,TRICALS,3.2986,0.2242,14.7095,0.0000,3.2986,2.8590,3.7381,True
2,Diagnostic_Delay,0.2833,0.1076,2.6339,0.0084,0.2833,0.0725,0.4942,True
3,Vital_capacity,0.1530,0.0974,1.5703,0.1163,0.1530,-0.0380,0.3439,False
4,Onset_Limb,0.9725,0.2870,3.3879,0.0007,0.9725,0.4099,1.5351,True
5,Sex_Male,1.0181,0.3393,3.0008,0.0027,1.0181,0.3531,1.6830,True
6,Sex_onset,-1.3339,0.3920,-3.4026,0.0007,-1.3339,-2.1023,-0.5655,True


In [ ]:
df_coef = extract_coefficients_from_fp_model_haz_scale(fp_model)
df_coef = round(df_coef, 4)
df_coef

In [ ]:
forest_plot_fp_model_haz_scale(extract_coefficients_from_fp_model_haz_scale(fp_model))

In [ ]:
# train_df.columns = train_df.columns.str.replace(' ', '_')
# test_df.columns = test_df.columns.str.replace(' ', '_')
ril_3010.columns

In [ ]:
means = ["Age", "TRICALS", "Diagnostic_Delay"]
modes = ["Sex_Male", "Onset_Limb", "Sex_onset",'Study_Arm_Placebo']

In [ ]:
# Prepare newdata for train and test
times_train = np.linspace(0, train_df["Disease_Duration"].max(), 100)
newdata_train = make_newdata_for_fp_model_prediction(train_df, times_train, means, modes, 'Disease_Duration')

times_test = np.linspace(0, test_df["Disease_Duration"].max(), 100)
newdata_test = make_newdata_for_fp_model_prediction(test_df, times_test, means, modes, 'Disease_Duration')

times_miroli0 = np.linspace(0, miroli0["Disease_Duration"].max(), 100)
newdata_miroli0 = make_newdata_for_fp_model_prediction(miroli0, times_miroli0, means, modes, 'Disease_Duration')

times_ril_3010 = np.linspace(0, ril_3010["Disease_Duration"].max(), 100)
newdata_ril_3010 = make_newdata_for_fp_model_prediction(ril_3010, times_ril_3010, means, modes, 'Disease_Duration')

times_proact0 = np.linspace(0, proact0["Disease_Duration"].max(), 100)
newdata_proact0 = make_newdata_for_fp_model_prediction(proact0, times_proact0, means, modes, 'Disease_Duration')

So while you can generate predictions at 50 months, you cannot validate them.

Prediction may be possible. Validation is not.


If you pretend the second trial has 50 months of follow-up:

You create informative censoring

You inflate uncertainty

You risk misleading conclusions about long-term survival

You can say:

“The model was trained on a 50-month trial and externally validated on an independent 22-month trial. Model performance was assessed at clinically relevant time points up to 22 months.”

In [ ]:
# ril_3010['Disease_Duration'].describe()
# miroli0['Disease_Duration'].describe()
# proact0['Disease_Duration'].describe()

In [ ]:
newdata_miroli0
newdata_ril_3010
# newdata_train

In [ ]:
# import inspect
# inspect.signature(prediction_survival_for_average_patient_fp_model_train_test)

In [ ]:
prediction_survival_for_average_patient_fp_model_train_test(train_df,
                                                            test_df,
                                                            ril_3010,
                                                            fp_model,
                                                            newdata_train=newdata_train,
                                                            newdata_test=newdata_test,
                                                            newdata_valid=newdata_ril_3010,
                                                            time_col='Disease_Duration',
                                                            title_suffix="title_suffix",
                                                            Train_set='Train_set',
                                                            Test_set='Test_set',
                                                            Validation_set='Validation_set')

In [ ]:
newdata_0_train, newdata_1_train = make_newdata_category_for_fp_model_prediction(train_df, means, modes, time_col="Disease_Duration", cat_col='Study_Arm_Placebo')
newdata_0_test, newdata_1_test = make_newdata_category_for_fp_model_prediction(test_df, means, modes, time_col="Disease_Duration", cat_col='Study_Arm_Placebo')
newdata_0_valid, newdata_1_valid = make_newdata_category_for_fp_model_prediction(ril_3010, means, modes, time_col="Disease_Duration", cat_col='Study_Arm_Placebo')

In [ ]:
prediction_survival_for_average_patient_cat_cov_fp_model_train_test(df_train=train_df,
                                                                    df_test=test_df,
                                                                    df_valid=ril_3010,
                                                                    fp_model=fp_model,
                                                                    newdata_0_train=newdata_0_train,
                                                                    newdata_1_train=newdata_1_train,
                                                                    newdata_0_test=newdata_0_test,
                                                                    newdata_1_test=newdata_1_test,
                                                                    newdata_0_valid=newdata_0_valid,
                                                                    newdata_1_valid=newdata_1_valid,
                                                                    time_col="Disease_Duration",
                                                                    title_suffix="",
                                                                    Train_set="",
                                                                    Test_set="",
                                                                    Validation_set=""
                                                                )

In [ ]:
prediction_hazard_for_average_patient_fp_model_train_test(
    train_df,
    test_df,
    ril_3010,
    fp_model,
    newdata_train=newdata_train,
    newdata_test=newdata_test,
    newdata_valid=newdata_ril_3010,
    time_col='Disease_Duration',
    title_suffix="title_suffix",
    Train_set='Train_set',
    Test_set='Test_set',
    Validation_set='Validation_set')

In [ ]:
prediction_hazard_for_average_patient_cat_cov_fp_model_train_test(df_train=train_df,
                                                                    df_test=test_df,
                                                                    df_valid=ril_3010,
                                                                    fp_model=fp_model,
                                                                    newdata_0_train=newdata_0_train,
                                                                    newdata_1_train=newdata_1_train,
                                                                    newdata_0_test=newdata_0_test,
                                                                    newdata_1_test=newdata_1_test,
                                                                    newdata_0_valid=newdata_0_valid,
                                                                    newdata_1_valid=newdata_1_valid,
                                                                    time_col="Disease_Duration",
                                                                    title_suffix="",
                                                                    Train_set="",
                                                                    Test_set="",
                                                                    Validation_set=""
                                                                )

## Prognostic Scores for Baseline Model (fit)

In [ ]:
train_df = prognostic_score(train_df, fp_model, groups=3)
train_df.head(3)

In [ ]:
train_df['risk_group'] = pd.qcut(train_df['prognostic_score'], 3, labels=['Low','Medium','High'])

train_df.head(6)
train_df.risk_group.value_counts().to_dict()

In [ ]:
for group in ['Low','Medium','High']:
    subset = train_df[train_df['risk_group']==group]
    kmf = KaplanMeierFitter()
    kmf.fit(subset['Disease_Duration'], subset['Event'], label=group)
    kmf.plot_survival_function()
plt.title("K–M - Prognostic Score")
plt.xlabel("Time")
plt.ylabel("Survival Probability")
plt.legend(title="Risk Group")
plt.grid(True)
plt.show()

## ***Simulate Virtual Placebo***

In [ ]:
train_df.info()

In [ ]:
cols = ['subject_id']

for col in (train_df.select_dtypes(include=['object', 'category', 'int']).columns):
    if col not in cols:
        print(f"{train_df[col].value_counts() / train_df[col].value_counts().sum()}\n")

In [ ]:
ons = train_df['Onset_Limb'].value_counts() / train_df['Onset_Limb'].value_counts().sum()
sex = train_df['Sex_Male'].value_counts() / train_df['Sex_Male'].value_counts().sum()
pla = train_df['Study_Arm_Placebo'].value_counts() / train_df['Study_Arm_Placebo'].value_counts().sum()
# exp = train_df['Expt_mirocals'].value_counts() / train_df['Expt_mirocals'].value_counts().sum()
# eur = train_df['European_Yes'].value_counts() / train_df['European_Yes'].value_counts().sum()


In [ ]:
ons, sex

In [ ]:
train_df.head(3)

In [ ]:
train_df.columns

In [ ]:
train_df.head(3)

In [ ]:
corr = train_df[['Age','Study_Arm_Placebo','Diagnostic_Delay','Vital_capacity','Sex_Male', 
            	 'Onset_Limb','TRICALS']].corr()
corr

In [ ]:
df = df_coef.copy()
df

In [ ]:
# df.loc[df["Variable"] == "Age", "Estimate"].values[0]
# ons
# sex[1]
# pla[1]

In [ ]:
var_specs_real = [
    # continuous
    {'name':'Age','type':'continuous','dist':{'kind':'normal','mean':0,'sd':1},'coef':df.loc[df["Variable"] == "Age", "Estimate"].values[0]},
    {'name':'TRICALS','type':'continuous','dist':{'kind':'normal','mean':0,'sd':1},'coef':df.loc[df["Variable"] == "TRICALS", "Estimate"].values[0]},
    {'name':'Diagnostic_Delay','type':'continuous','dist':{'kind':'normal','mean':0,'sd':1},
                                                           'coef':df.loc[df["Variable"] == "Diagnostic_Delay", "Estimate"].values[0]},
    # {'name':'Vital_capacity','type':'continuous','dist':{'kind':'normal','mean':0,'sd':1},
    #                                                        'coef':df.loc[df["Variable"] == "Vital capacity", "Estimate"].values[0]},

    # binary
    {'name':'Sex_Male','type':'binary','prob':sex[1],'coef':df.loc[df["Variable"] == "Sex_Male", "Estimate"].values[0]},
    {'name':'Onset_Limb','type':'binary','prob':ons[1],'coef':df.loc[df["Variable"] == "Onset_Limb", "Estimate"].values[0]},
    # {'name':'Expt_mirocals','type':'binary','prob':exp[1],'coef':df.loc[df["Variable"] == "Expt_mirocals", "Estimate"].values[0]},
    {'name':'Study_Arm_Placebo','type':'binary','prob':pla[1],'coef':df.loc[df["Variable"] == "Study_Arm_Placebo", "Estimate"].values[0]},

    # # categorical
    # {'name':'Onset_site','type':'categorical',
    #  'levels':['bulbar','limb','other'],'probs':[0.218484,0.746833,0.034683],
    #  'coef':{'limb':1.151159,'other':2.508786},'ref':'bulbar'}
]

corr = train_df[['Age', 'TRICALS', 'Diagnostic_Delay','Sex_Male', 'Onset_Limb', 'Study_Arm_Placebo']].corr()

# df_sim = simulate_cox_dataset(n=df1.shape[0], baseline='weibull', lam=0.01, rho=1.5, 
#                               censor_rate=0.3, seed=42, corr=corr, var_specs=var_specs_real)


# df_sim.head(3)

In [ ]:
train_df.Event.value_counts() / train_df.Event.value_counts().sum()

In [ ]:
H0_df = estimate_baseline_cumhaz(train_df, time_col='Disease_Duration', event_col='Event')
# H0_df

In [ ]:
# Real dataset
# H0_df = estimate_baseline_cumhaz(real_df)

# Simulated dataset
sim_df = simulate_cox_dataset_with_correlation(
    # n=500,
    n=train_df.shape[0],
    var_specs=var_specs_real,
    corr=corr,
    baseline="empirical",
    H0_df=H0_df,
    censor_rate=0.4,
    seed=42
)


In [ ]:
sim_df.head(3)

In [ ]:
# Create interactions
sim_df['Sex_onset'] = sim_df['Sex_Male']*sim_df['Onset_Limb']
sim_df.head(3)

In [ ]:
sim_df.event.value_counts() #/ sim_df.event.value_counts().sum()

In [ ]:
train_df.head(3)

In [ ]:
train_df.Event.value_counts() #/ train_df.event.value_counts().sum()

In [ ]:
ons = train_df['Onset_Limb'].value_counts() / train_df['Onset_Limb'].value_counts().sum()
sex = train_df['Sex_Male'].value_counts() / train_df['Sex_Male'].value_counts().sum()
sex_ons = train_df['Sex_onset'].value_counts() / train_df['Sex_onset'].value_counts().sum()
print(ons, sex, sex_ons)

print("\n---- Simulated data ----")
ons1 = sim_df['Onset_Limb'].value_counts() / sim_df['Onset_Limb'].value_counts().sum()
sex1 = sim_df['Sex_Male'].value_counts() / sim_df['Sex_Male'].value_counts().sum()
sex_ons1 = sim_df['Sex_onset'].value_counts() / sim_df['Sex_onset'].value_counts().sum()

print(ons1, sex1, sex_ons1)

In [ ]:
with conversion.localconverter(pandas2ri.converter):
    r_df_sim = pandas2ri.py2rpy(sim_df)

In [ ]:
# # Define the formula
formula1 = Formula('Surv(true_survival_time, event==1) ~ Age + Study_Arm_Placebo + Onset_Limb + TRICALS + Sex_Male + Diagnostic_Delay')

# Fit the flexible parametric model
# df = degrees of freedom for spline, adjust as needed
fp_model1 = rstpm2.stpm2(formula1, data=r_df_sim, df=0)
# print(ro.r.summary(fp_model1))

In [ ]:
df_coef1 = extract_coefficients_from_fp_model_haz_scale(fp_model1)
df_coef1

In [ ]:
df_coef

In [ ]:
coefs = pd.merge(df_coef, df_coef1, on="Variable", suffixes=('_real', '_simulated'))
coefs = round(coefs, 4)
coefs

In [ ]:
coefs1 = coefs[['Variable', 'Estimate_real', 'Estimate_simulated']]
coefs1

In [ ]:
x = np.arange(len(coefs1))      # positions
width = 0.35                # bar width

plt.figure(figsize=(8, 4))

# Bars
plt.bar(x - width/2, coefs1["Estimate_real"], width, label="Original Data")
plt.bar(x + width/2, coefs1["Estimate_simulated"], width, label="Simulated Data")

# Zero line
plt.axhline(0, linestyle="--", linewidth=1)

# Labels
plt.xticks(x, coefs1["Variable"], rotation=45, ha="right")
plt.ylabel("Coefficient value")
plt.title("Bar Plot of Coefficients: Original vs Simulated Data")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
coefs1 = coefs[['Variable', 'HR_real', 'HR_simulated']]
coefs1

x = np.arange(len(coefs1))      # positions
width = 0.35                # bar width

plt.figure(figsize=(8, 4))

# Bars
plt.bar(x - width/2, coefs1["HR_real"], width, label="Original Data")
plt.bar(x + width/2, coefs1["HR_simulated"], width, label="Simulated Data")

# Zero line
plt.axhline(0, linestyle="--", linewidth=1)

# Labels
plt.xticks(x, coefs1["Variable"], rotation=45, ha="right")
plt.ylabel("Coefficient value")
plt.title("Bar Plot of Coefficients: Original vs Simulated Data")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
for col in ['Age', 'TRICALS', 'Diagnostic_Delay']:  # replace with your continuous columns
    plt.figure(figsize=(5,4))
    plt.hist(train_df[col], bins=30, alpha=0.5, label='Original', density=True)
    plt.hist(sim_df[col], bins=30, alpha=0.5, label='Simulated', density=True)
    plt.title(f'Distribution of {col}')
    plt.legend()
    plt.show()

In [ ]:
cols = ['Age', 'TRICALS', 'Diagnostic_Delay']

fig, axes = plt.subplots(1, 2, figsize=(12,5))

# Original correlation
sns.heatmap(train_df[cols].corr(), annot=True, cmap="coolwarm", ax=axes[0], vmin=-1, vmax=1)
axes[0].set_title("Original Correlation Matrix")

# Simulated correlation
sns.heatmap(sim_df[cols].corr(), annot=True, cmap="coolwarm", ax=axes[1], vmin=-1, vmax=1)
axes[1].set_title("Simulated Correlation Matrix")

plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import ttest_ind

cols = ['Age', 'TRICALS', 'Diagnostic_Delay']

results = []

for col in cols:
    original = train_df[col].dropna()
    simulated = sim_df[col].dropna()

    # Welch t-test
    t_stat, p_val = ttest_ind(original, simulated, equal_var=False)

    # Mean difference
    mean_diff = simulated.mean() - original.mean()

    # Standard errors for CI (Welch)
    se = np.sqrt(original.var()/len(original) + simulated.var()/len(simulated))

    # 95% CI
    ci_low = mean_diff - 1.96 * se
    ci_high = mean_diff + 1.96 * se

    results.append([col, mean_diff, ci_low, ci_high, p_val])

df_plot = pd.DataFrame(results, columns=["Variable", "MeanDiff", "CI_low", "CI_high", "p_value"])


# --------- Forest Plot ---------
plt.figure(figsize=(8, 6))

y_pos = np.arange(len(df_plot))

plt.errorbar(
    df_plot["MeanDiff"], 
    y_pos, 
    xerr=[df_plot["MeanDiff"] - df_plot["CI_low"], df_plot["CI_high"] - df_plot["MeanDiff"]],
    fmt='o', 
    capsize=5
)

plt.axvline(0, color='gray', linestyle='--')

plt.yticks(y_pos, df_plot["Variable"])
plt.xlabel("Mean Difference (Simulated − Original)")
plt.title("Forest Plot of T-Test Mean Differences")

plt.tight_layout()
plt.show()

In [ ]:
# remove the interaction term from the model